In [4]:
import pandas as pd
import json

df = pd.read_parquet("transactions.parquet")
df['year'] = df['Order_Date_FK'].astype(str).str[:4]
df['month'] = df['Order_Date_FK'].astype(str).str[4:6]
df['day'] = df['Order_Date_FK'].astype(str).str[6:]

# values = dict()

# for col in df.columns:
#     values[col] = list(df[col].astype(str).unique())

# values

{'Shop': ['36',
  '58',
  '49',
  '15',
  '57',
  '80',
  '8',
  '83',
  '64',
  '76',
  '14',
  '67',
  '46',
  '6',
  '73',
  '32',
  '68',
  '1',
  '60',
  '28',
  '21',
  '43',
  '47',
  '24',
  '9',
  '33',
  '61',
  '48',
  '11',
  '75',
  '7',
  '81',
  '19',
  '18',
  '79',
  '86',
  '56',
  '63',
  '82',
  '84',
  '5',
  '31',
  '71',
  '25',
  '85',
  '78',
  '3',
  '44',
  '52',
  '12',
  '13',
  '65',
  '45',
  '20',
  '66',
  '10',
  '55',
  '23',
  '2',
  '77',
  '37',
  '29',
  '59',
  '50',
  '72',
  '69',
  '74',
  '4',
  '70',
  '62',
  '22',
  '34',
  '53',
  '30',
  '51',
  '35',
  '26',
  '27',
  '38',
  '41',
  '54',
  '16',
  '42',
  '102',
  '103'],
 'Order_Date_FK': ['20210730',
  '20210731',
  '20210801',
  '20210802',
  '20210803',
  '20210804',
  '20210805',
  '20210806',
  '20210807',
  '20210808',
  '20210809',
  '20210810',
  '20210811',
  '20210812',
  '20210813',
  '20210814',
  '20210815',
  '20210601',
  '20210602',
  '20210603',
  '20210604',
  '2021

In [5]:
with open("values.json", "w") as f:
    json.dump(values, f)

In [6]:
import sqlite3 as sq
import pandas as pd

connection = sq.connect("database.db")

#### Duplicates are unclear. It would be better to know why there are duplicates, and be certain how to handle them correctly.

In [7]:
duplicates = pd.read_sql_query(
"""
SELECT *
FROM trans
WHERE SaleDocumentNumber = 23719501
""", connection
)
duplicates

,index,Shop,Order_Date_FK,ProductCode,OriginalSaleAmountInclVAT,CustomerID,SaleDocumentNumber,Returned,RevenueInclVAT,CostPriceExclVAT,BrandName,ModelGroup,ProductGroup
0,1005963,49,20210701,-5223360592088065790,105.00,-1294657933607279032,23719501,0,10.50,25.608496,1168115093187640245,-4124706989267335995,-453682476182549203
1,1005964,49,20210701,3510254952241336477,75.95,-1294657933607279032,23719501,0,7.60,24.500000,33538854880025124,-4124706989267335995,-453682476182549203
2,1005965,49,20210701,449823314919264450,59.95,-1294657933607279032,23719501,0,6.00,18.950000,9178759879362980169,-4124706989267335995,-453682476182549203
3,1005966,49,20210701,3274348471310059289,170.00,-1294657933607279032,23719501,0,17.00,62.036017,-424472638565217610,-4124706989267335995,-453682476182549203
4,1005967,49,20210701,-5584415452890875547,130.00,-1294657933607279032,23719501,0,13.00,36.000000,-7177370704634981598,-4124706989267335995,-453682476182549203
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1349146,49,20210701,-2854091718625257153,65.95,-1294657933607279032,23719501,0,6.59,25.714611,2776326510072975976,-4124706989267335995,-453682476182549203
98,1349147,49,20210701,-8041448267277774414,75.95,-1294657933607279032,23719501,0,7.59,17.493915,1168115093187640245,-4124706989267335995,-453682476182549203
99,1349148,49,20210701,-7462286497937572848,175.00,-1294657933607279032,23719501,0,17.50,64.152000,-1873787772202565890,-4124706989267335995,-453682476182549203
100,1349149,49,20210701,441949993592162707,130.00,-1294657933607279032,23719501,0,13.00,46.889754,1857918515453071333,-4124706989267335995,-453682476182549203


In [8]:
duplicates = pd.read_sql_query(
"""
SELECT
    COUNT(ProductCode) AS uniques
FROM trans
WHERE SaleDocumentNumber = 23719501
GROUP BY ProductCode
ORDER BY uniques DESC
""", connection
)
duplicates.head(10)

,uniques
0,4
1,4
2,4
3,4
4,2
5,2
6,2
7,2
8,2
9,2


#### Now let's take a look at returns in various ways.

In [9]:
df["Returned"].sum()

67795

In [10]:
returns = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    ProductCode, BrandName,
    OriginalSaleAmountInclVAT
FROM trans
GROUP BY ProductCode
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    ProductCode, BrandName, OriginalSaleAmountInclVAT
FROM a
ORDER BY TotalReturns DESC, PercentReturned DESC
""", connection)

print("\tGrouped by PoductCode, PercentReturned > 0.2, ordered by TotalReturns descending")
returns[returns["PercentReturned"] > 0.2][:10]

	Grouped by PoductCode, PercentReturned > 0.2, ordered by TotalReturns descending


,PercentReturned,TotalItems,TotalReturns,ProductCode,BrandName,OriginalSaleAmountInclVAT
0,0.326376,1054,344,-3711611854324721442,6164254645951948400,45.95
11,0.268102,511,137,4754863614336515610,-1671135878886367730,25.95
14,0.210708,579,122,5587091866033151813,3147597281999378922,15.95
28,0.321070,299,96,4101533479629837146,-3734237314099258167,75.95
37,0.231405,363,84,-3103717362647513544,-8710939345520521313,39.95
38,0.432292,192,83,4664267319375427392,-8846961206607438497,75.95
58,0.250923,271,68,4510119518088610237,6164254645951948400,69.95
98,0.412214,131,54,8706358678450957216,-1671135878886367730,27.95
110,0.481481,108,52,1688912043498932481,-8507421977918128293,95.95
159,0.200913,219,44,-8680557187301822340,-1873787772202565890,75.95


#### It is quite possible that product with ProductCode -3711611854324721442 is bad.

In [11]:
returns = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    ModelGroup
FROM trans
GROUP BY ModelGroup
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    ModelGroup
FROM a
ORDER BY TotalReturns DESC, PercentReturned DESC
""", connection)

print("\tGrouped by ModelGroup, PercentReturned > 0.2, ordered by TotalReturns descending")
returns[:10]

	Grouped by ModelGroup, PercentReturned > 0.2, ordered by TotalReturns descending


,PercentReturned,TotalItems,TotalReturns,ModelGroup
0,0.054149,564852,30586,3162564956579801398
1,0.037423,703042,26310,-4124706989267335995
2,0.044297,64497,2857,-6715310361529623819
3,0.039762,38580,1534,-1330987452709117740
4,0.020803,71818,1494,-248459121805647149
5,0.036640,34771,1274,8676381074854750838
6,0.060680,20188,1225,-1172740817228586022
7,0.035239,14728,519,-965590225519141271
8,0.015010,21986,330,-2375149457939821456
9,0.037417,8285,310,-3170699624988567198


In [12]:
returns = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    BrandName
FROM trans
GROUP BY BrandName
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    BrandName
FROM a
ORDER BY TotalReturns DESC, PercentReturned DESC
""", connection)

print("\tGrouped by BrandName, PercentReturned > 0.2, ordered by TotalReturns descending")
returns[returns["PercentReturned"] > 0.2][:10]

	Grouped by BrandName, PercentReturned > 0.2, ordered by TotalReturns descending


,PercentReturned,TotalItems,TotalReturns,BrandName
53,0.251429,1050,264,-8507421977918128293
98,0.249417,429,107,611049624476511346
136,0.221122,303,67,6862799060630055392
139,0.278481,237,66,8690259017905730373
140,0.216393,305,66,7788141470214192600
165,0.343066,137,47,-7130095634701368819
166,0.218605,215,47,-4606403254425245430
184,0.234568,162,38,-7921480381513708220
195,0.430556,72,31,4717096102405955547
204,0.333333,78,26,-7827463481178904503


#### Notice how BrandName -8507421977918128293 has 25% returns on 1000 items. That seems like a bad brand to me. Particularly their product 1688912043498932481 with 50% returns on around 100 items. Maybe it's worth looking into.

In [13]:
results = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    ProductCode
FROM trans
WHERE BrandName = -8507421977918128293
GROUP BY ProductCode
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    ProductCode
FROM a
ORDER BY TotalReturns DESC, PercentReturned DESC
""", connection)

print("\tGrouped by PoductCode, PercentReturned > 0.2, ordered by TotalReturns descending")
results[results["PercentReturned"] > 0.][:10]

	Grouped by PoductCode, PercentReturned > 0.2, ordered by TotalReturns descending


,PercentReturned,TotalItems,TotalReturns,ProductCode
0,0.481481,108,52,1688912043498932481
1,0.306452,62,19,-2589790336782400676
2,0.410256,39,16,-575353219603877387
3,0.238095,63,15,-4297715029242917879
4,0.279070,43,12,2104612900236091154
5,0.184615,65,12,8667661596481621898
6,0.277778,36,10,7884848769945646548
7,0.263158,38,10,-3515954374243214119
8,0.346154,26,9,-1107135324166745045
9,0.166667,54,9,-5774003379652257129


# So what about shops?

In [31]:
returns_per_shop = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    Shop
FROM trans
GROUP BY Shop
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    Shop
FROM a
ORDER BY PercentReturned DESC
""", connection)

print("\tGrouped by Shop, ordered by PercentReturned descending")
returns_per_shop[:10]

	Grouped by Shop, ordered by PercentReturned descending


,PercentReturned,TotalItems,TotalReturns,Shop
0,0.186977,140456,26262,102
1,0.164588,8901,1465,103
2,0.064000,125,8,41
3,0.039513,21512,850,30
4,0.037832,22838,864,68
5,0.035541,11930,424,33
6,0.033156,18820,624,6
7,0.031596,14812,468,43
8,0.031090,18720,582,47
9,0.029993,23072,692,70


# Any customers of interest?

In [15]:
returns_per_customer = pd.read_sql_query(
"""
WITH a AS (
SELECT
    COUNT(Returned) AS C,
    SUM(Returned) AS S,
    CustomerID
FROM trans
GROUP BY CustomerID
ORDER BY C DESC
)
SELECT
    CAST(S AS FLOAT) / C AS PercentReturned,
    C AS TotalItems,
    S AS TotalReturns,
    CustomerID,
    S / 67795.0 AS PercentOfAllReturns
FROM a
ORDER BY TotalReturns DESC, PercentReturned DESC
""", connection)

print("\tGrouped by CustomerID, ordered by TotalReturns and PercentReturned descending")
returns_per_customer

	Grouped by CustomerID, ordered by TotalReturns and PercentReturned descending


,PercentReturned,TotalItems,TotalReturns,CustomerID,PercentOfAllReturns
0,0.143486,54277,7788,-5287955982418449235,0.114876
1,0.129367,3950,511,1727613839816903509,0.007537
2,0.029466,7602,224,4572026730515361595,0.003304
3,0.022233,7961,177,-4279123115125507326,0.002611
4,0.021077,7022,148,6455628090489614628,0.002183
...,...,...,...,...,...
347857,0.000000,1,0,-9217944974532867625,0.000000
347858,0.000000,1,0,-9219368559219805324,0.000000
347859,0.000000,1,0,-9219371575752114423,0.000000
347860,0.000000,1,0,-9221298982193402923,0.000000


#### Customer -5287955982418449235 is pretty huge, and responsible for 11% of all returns.